## Формирование общего датасета транзакций

### Слияние данных

Сольем данные всех файлов в один сет.

In [5]:
import pandas as pd
pd.set_option('max_colwidth', 400)


# открываем файлы
tr_mcc_codes = pd.read_csv('datasets/tr_mcc_codes_featured.csv', delimiter=',')
tr_types = pd.read_csv('datasets/tr_types_featured.csv', delimiter=',')
transactions = pd.read_csv('kaggle_datasets/transactions.csv.zip', delimiter=',')
gender_train = pd.read_csv('kaggle_datasets/gender_train.csv', delimiter=',')


# сливаем файлы
rawset = pd.merge(transactions, gender_train, on='customer_id', how='left')
rawset = pd.merge(rawset, tr_types, on='tr_type', how='left')
rawset = pd.merge(rawset, tr_mcc_codes, on='mcc_code', how='left')

In [6]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group
4852146,24634142,409 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54
3368757,4891744,372 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54
6191439,29894123,411 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60
4518851,41783890,453 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60
5414347,56680038,344 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54


Для подготовки сета необходимо дополнительно обработать поля времени операций `tr_datetime` и размера `amount`.

Размер операции `amount` будет удобно перевести в абсолютные значения с пометкой - расход это или доход. Это позволит работать с валовыми объемами операций вне зависимости от их направления.

Значение `tr_datetime` необходимо привести к формату даты-времени, желательно с корректной привязкой к временной шкале.

### Обработка поля `amount`

In [7]:
# Добавим абсолютное значение поля как `amount_abs`:
rawset['amount_abs'] = rawset['amount'].apply(abs)

# Добавим направление транзакции, введя поле is_income. Если это входящая тразнакция - ставим 1, если исходящая (расход) - 0.
rawset.loc[rawset['amount']>0, 'is_income'] = 1
rawset.loc[rawset['amount']<0, 'is_income'] = 0

In [8]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income
4852146,24634142,409 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,9515.95,0.0
3368757,4891744,372 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,875.91,0.0
6191439,29894123,411 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,49499.98,1.0
4518851,41783890,453 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,33688.74,1.0
5414347,56680038,344 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,19512.52,0.0


Теперь у нас есть поля `amount_abs` и `is_income`. Это позволит нам удобно собирать общие агрегаты по операциям клиента.

### Обработка поля `tr_datetime`

#### Приведение к формату даты-времени

In [9]:
# Приводим значения поля `tr_datetime` к формату даты-времени. Пишем функцию и пробуем ее запустить.

import datetime
def timeformatter(rawdate):
    # разделяем поле на два значения (разделитель - пробел)
    rows = rawdate.split(' ')
    
    # первое значение преобразуем в дату, второе - в формат времени
    days = datetime.timedelta(days=int(rows[0]))
    time = datetime.datetime.strptime(rows[1], '%H:%M:%S')

    return (days+time)


try:
    rawset['tr_datetime'] = rawset['tr_datetime'].apply(timeformatter)

except ValueError as err:
    print('Ошибка: ', err)

Ошибка:  second must be in 0..59


Судя по ошибке, поле содержит данные со значением секунд большим чем 59. 

Проверим это: выведем значения, в которых количество секунд больше 59.

In [10]:
# Функция выдает ошибку, потому что почти 100 000 записей имеют количество секунд больше 59

rawset[rawset['tr_datetime'].apply(lambda x: int(x[-2::]) > 59)]['tr_datetime']

77          43 06:27:60
221        123 07:16:60
233        132 13:24:60
292        154 11:02:60
302        157 11:55:60
               ...     
6848953    115 11:44:60
6848980    143 07:00:60
6849179    359 16:47:60
6849257    420 19:06:60
6849342    454 10:54:60
Name: tr_datetime, Length: 99524, dtype: object

Таких значений почти 100 000. Судя по тому, что визуально они все равны 60, это ошибка округления. 

Приведем значение более 59 секунд к 59 и повторим обработку.

In [11]:
# Исправляем секунды: приводим 60 секунд к значению 59, проверяем есть ли записи со значением секунд больше 59.

def seconds_fixer(string):
    first_part = string[:-2:] # выделяем все до секунд
    last_2chrs = string[-2::] # выделяем секунды

    if int(last_2chrs) > 59:
        last_2chrs = '59'
    
    return (first_part+last_2chrs)

rawset['tr_datetime'] = rawset['tr_datetime'].apply(seconds_fixer)


# Снова пробуем преобразовать строку в формат даты-времени
rawset['tr_datetime'] = rawset['tr_datetime'].apply(timeformatter)


# добавляем отдельно поле даты
rawset['tr_date'] = rawset['tr_datetime'].apply(lambda x: x.date())


In [12]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income,tr_date
4852146,24634142,1901-02-14 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,9515.95,0.0,1901-02-14
3368757,4891744,1901-01-08 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,875.91,0.0,1901-01-08
6191439,29894123,1901-02-16 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,49499.98,1.0,1901-02-16
4518851,41783890,1901-03-30 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,33688.74,1.0,1901-03-30
5414347,56680038,1900-12-11 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,19512.52,0.0,1900-12-11


Мы получили значения `tr_datetime` в корректном формате даты-времени и выделили дату (`tr_date`) для большего удобства.

Сохраняем сет в файл.

In [13]:
rawset.to_pickle('datasets/trx_set_no_datelink.pkl')

Далее попробуем найти истинные значения дат.